# BERT Fine Tuning

https://github.com/rasbt/machine-learning-book/blob/main/ch16/ch16-part3-bert.ipynb

In [30]:
try:
    import transformers
except:
    !pip install transformers==4.9.1 -q
    import transformers

import gzip
import shutil
import time
import pandas as pd
import numpy as np
import requests
import torch
import torch.nn.functional as F
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda


In [2]:
torch.backends.cudnn.deterministic = True
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

NUM_EPOCHS = 1

In [3]:
url = ("https://github.com/rasbt/machine-learning-book/raw/main/ch08/movie_data.csv.gz")
filename = url.split("/")[-1]

with open(filename, "wb") as f:
    r = requests.get(url)
    f.write(r.content)

with gzip.open(filename, "rb") as f_in:
    with open(filename[:-3], "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

In [4]:
df = pd.read_csv(filename[:-3])
df.head()

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


In [5]:
ind0 = 35000
ind1 = 40000
train_texts = df.iloc[:ind0]["review"].values
train_labels = df.iloc[:ind0]["sentiment"].values

valid_texts = df.iloc[ind0:ind1]["review"].values
valid_labels = df.iloc[ind0:ind1]["sentiment"].values

test_texts = df.iloc[ind1:]["review"].values
test_labels = df.iloc[ind1:]["sentiment"].values

In [6]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

train_encodings = tokenizer(list(train_texts), truncation = True, padding = True)
valid_encodings = tokenizer(list(valid_texts), truncation = True, padding = True)
test_encodings = tokenizer(list(test_texts), truncation = True, padding = True)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [7]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [8]:
train_dataset = IMDbDataset(train_encodings, train_labels)
valid_dataset = IMDbDataset(valid_encodings, valid_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 16, shuffle = True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size = 16, shuffle = False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 16, shuffle = False)

In [9]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
model.to(DEVICE)
model.train()

optim = torch.optim.Adam(model.parameters(), lr = 5e-5)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

In [11]:
def compute_accuracy(model, data_loader, device):
    with torch.no_grad():
        correct_pred, num_examples = 0, 0
        for batch_idx, batch in enumerate(data_loader):
            ### Prepare data
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs['logits']
            predicted_labels = torch.argmax(logits, 1)
            num_examples += labels.size(0)
            correct_pred += (predicted_labels == labels).sum()
        
        return correct_pred.float() / num_examples * 100

In [14]:
def predict(model, data_loader, device, return_X = False):
    y = []
    if return_X == True:
        X = []

    with torch.no_grad():
        for input_ids, batch in enumerate(data_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask = attention_mask)
            logits = outputs['logits']
            predicted_labels = torch.argmax(logits, 1)
            y.append(predicted_labels)
            if return_X == True:
                X.append(batch)
                
    if return_X == True:
        return X, y
    else:
        return y

In [13]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch in enumerate(train_loader):
        ### Prepare data
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)

        ### Forward
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss, logits = outputs['loss'], outputs['logits']
        
        ### Backward
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        ### Logging
        if not batch_idx % 250:
            print (f'Epoch: {epoch+1:04d}/{NUM_EPOCHS:04d} | '
                   f'Batch {batch_idx:04d}/{len(train_loader):04d} | '
                   f'Loss: {loss:.4f}')
            
    model.eval()
    with torch.set_grad_enabled(False):
        print(f'Training accuracy: '
              f'{compute_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nValid accuracy: '
              f'{compute_accuracy(model, valid_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')

Epoch: 0001/0001 | Batch 0000/2188 | Loss: 0.7671
Epoch: 0001/0001 | Batch 0250/2188 | Loss: 0.1177
Epoch: 0001/0001 | Batch 0500/2188 | Loss: 0.6105
Epoch: 0001/0001 | Batch 0750/2188 | Loss: 0.1122
Epoch: 0001/0001 | Batch 1000/2188 | Loss: 0.2796
Epoch: 0001/0001 | Batch 1250/2188 | Loss: 0.3082
Epoch: 0001/0001 | Batch 1500/2188 | Loss: 0.0764
Epoch: 0001/0001 | Batch 1750/2188 | Loss: 0.1930
Epoch: 0001/0001 | Batch 2000/2188 | Loss: 0.0852
Training accuracy: 96.57%
Valid accuracy: 92.62%
Time elapsed: 40.34 min
Total Training Time: 40.34 min
Test accuracy: 92.90%


In [17]:
X_pred, y_pred = predict(model, test_loader, DEVICE, True)

In [20]:
X_pred[0]

{'input_ids': tensor([[  101,  2107,  2943,  ...,     0,     0,     0],
         [  101,  2021,  2726,  ...,     0,     0,     0],
         [  101,  2339,  2023,  ...,     0,     0,     0],
         ...,
         [  101,  1037,  2177,  ...,     0,     0,     0],
         [  101,  2054, 10239,  ...,     0,     0,     0],
         [  101,  1045,  3342,  ...,     0,     0,     0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'labels': tensor([1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1])}

In [21]:
y_pred[0]

tensor([1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1], device='cuda:0')